# PROJECT OBJECTIVE: 
To Build a recommendation system using popularity based and collaborative filtering methods to recommend 
mobile phones to a user which are most popular and personalised respectively

1. Import the necessary libraries and read the provided CSVs as a data frame and perform the below steps.

A. Merge all the provided CSVs into one dataFrame.

In [1]:
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans
from surprise import accuracy
from surprise import Reader
import os


import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
import pandas as pd

df1 = pd.read_csv('C:\\Users\\vrooky-montzo\\Downloads\\Data Set\\Data Set\\phone_user_review_file_1.csv', encoding='latin-1')
df2 = pd.read_csv('C:\\Users\\vrooky-montzo\\Downloads\\Data Set\\Data Set\\phone_user_review_file_2.csv', encoding='latin-1')
df3 = pd.read_csv('C:\\Users\\vrooky-montzo\\Downloads\\Data Set\\Data Set\\phone_user_review_file_3.csv', encoding='latin-1')
df4 = pd.read_csv('C:\\Users\\vrooky-montzo\\Downloads\\Data Set\\Data Set\\phone_user_review_file_4.csv', encoding='latin-1')
df5 = pd.read_csv('C:\\Users\\vrooky-montzo\\Downloads\\Data Set\\Data Set\\phone_user_review_file_5.csv', encoding='latin-1')
df6 = pd.read_csv('C:\\Users\\vrooky-montzo\\Downloads\\Data Set\\Data Set\\phone_user_review_file_6.csv', encoding='latin-1')
df_all = [df1,df2,df3,df4,df5,df6]
df_phone = pd.concat(df_all)
df_phone

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
163832,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,Weil mein Onkel bei ALcatel arbeitet habe ich ...,david.paul,Alcatel Club Plus Handy
163833,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Hy Liebe Leserinnen und Leser!! Ich habe seit ...,Christiane14,Alcatel Club Plus Handy
163834,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,"Jetzt hat wohl Alcatell gedacht ,sie machen wa...",michaelawr,Alcatel Club Plus Handy
163835,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,Ich bin seit 2 Jahren (stolzer) Besitzer eines...,claudia0815,Alcatel Club Plus Handy


1B. Explore, understand the Data and share at least 2 observations. 

In [3]:
df_phone.shape

(1415133, 11)

In [4]:
df_phone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


In [5]:
#Data has float and object values.
#some variable column has null/missing values.

In [6]:
df_phone.describe()

,score,score_max
count,1.351644e+06,1351644.0
mean,8.007060e+00,10.0
std,2.616121e+00,0.0
min,2.000000e-01,10.0
25%,7.200000e+00,10.0
50%,9.200000e+00,10.0
75%,1.000000e+01,10.0
max,1.000000e+01,10.0


1.C.Round off scores to the nearest integers.

In [7]:
# rounding of values and storing in new column
df_phone['NewScore']=df_phone['score'].round()
df_phone['NewScore']

0         10.0
1         10.0
2          6.0
3          9.0
4          4.0
          ... 
163832     2.0
163833    10.0
163834     2.0
163835     8.0
163836     2.0
Name: NewScore, Length: 1415133, dtype: float64

In [8]:
df_phone['NewScore'].isnull().sum()

63489

In [9]:
df_phone['score_max'].isnull().sum()

63489

1D. Check for missing values. Impute the missing values, if any

In [10]:
df_phone.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
NewScore     63489
dtype: int64

In [11]:
cols = ["NewScore","author",'product']
df_phone[cols]=df_phone[cols].fillna(df_phone.median().iloc[0])

In [12]:
df_phone.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author           0
product          0
NewScore         0
dtype: int64

In [13]:
#since we are going to use just the three columns, viz, 'newscore', 'author', and 'product' for building recommendation engine 
#we go forward with just imputing those three columns.

1E. Check for duplicate values and remove them, if any

In [14]:
df_phone.duplicated().sum()

6412

In [15]:
df_phon = df_phone.drop_duplicates()

In [16]:
df_phon.duplicated().sum()

0

In [17]:
df_phon.shape

(1408721, 12)

1F. Keep only 1 Million data samples. Use random state=612.

In [18]:
data = df_phon.sample(n=1000000, random_state=612, axis=0)
data.head(20)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product,NewScore
347247,/cellphones/nokia-6301/,2/21/2011,en,gb,Dooyoo,dooyoo.co.uk,10.0,10.0,This phone has lasted the test of time and has...,gilay28,Nokia 6301,10.0
80368,/cellphones/apple-iphone-4s/,13/1/2012,en,us,Amazon,amazon.com,10.0,10.0,"For years, I've been using Android phones. Fro...",Stacy G.,"Apple iPhone 4S Verizon Cellphone, 16GB, White",10.0
46940,/cellphones/samsung-galaxy-note-3-neo-4g-n7505/,8/21/2014,de,de,Amazon,amazon.de,10.0,10.0,Habe mir das Samsung Galaxy Note 3 Neo als Nac...,BulldoZer,"Samsung Galaxy Note 3 Neo Smartphone (13,94 cm...",10.0
283355,/cellphones/nokia-5030/,4/12/2010,en,us,HandCellPhone,handcellphone.com,6.0,10.0,nokia 5030 me game kesa dalay,vinod kumar,Nokia 5030,6.0
61365,/cellphones/huawei-p9-plus/,8/2/2016,it,it,Amazon,amazon.it,10.0,10.0,"Beh, dopo Iphone e Lumia sono passato al mondo...",Giorgio,"Huawei P9 Plus Smartphone, LTE, Display 5.5'' ...",10.0
114568,/cellphones/zte-zmax/,1/11/2015,en,us,Amazon,amazon.com,8.0,10.0,The zmax is a fantastic phone! I'll tell you w...,Amazon Customer,ZTE Z Max 2 16GB Unlocked GSM 4G LTE Quad-Core...,8.0
74871,/cellphones/samsung-galaxy-s-iii-neo-i9300i/,9/1/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,"Gostei muito do produto. Fininho, lindo e bem ...",Rita Goes,Samsung Smartphone Samsung Galaxy S III Neo Du...,8.0
368948,/cellphones/samsung-galaxy-grand-prime/,6/3/2015,en,in,Amazon,amazon.in,6.0,10.0,Camera is good Quality,Amazon Customer,"Samsung Galaxy Grand Prime SM-G530H (White, 8GB)",6.0
105717,/cellphones/nokia-6170/,8/28/2007,es,uy,MercadoLibre Uruguay,opinion.mercadolibre.com.uy,8.0,10.0,"Durante los 2 aÃ±os que tube uno, me resulto m...",NEVORA,Nokia 6170,8.0
116453,/cellphones/lg-optimus-exceed-2/,5/30/2015,en,us,Amazon,amazon.com,10.0,10.0,I bought the LG optimus Exceed 2 to run perk T...,Lindsay Wong,LG Optimus Exceed 2 (Verizon Prepaid) (Discont...,10.0


1G. Drop irrelevant features. Keep features like Author, Product, and Score.

In [19]:
# we can drop phone_url,date,lang,country,source,domain and extract since they do not contribute in deciding popularity. 
phone_df = data.drop(['score','source','lang','score_max','extract','date','phone_url','country','domain'],axis = 1)
phone_df.head(2)

,author,product,NewScore
347247,gilay28,Nokia 6301,10.0
80368,Stacy G.,"Apple iPhone 4S Verizon Cellphone, 16GB, White",10.0


2A. Identify the most rated products.

In [20]:
grouped_multiple1 = phone_df.groupby(['product']).NewScore.agg(['count']).sort_values(by=['count'], ascending=False)
grouped_multiple1.head(20)

,count
product,
"Lenovo Vibe K4 Note (White,16GB)",3700
"Lenovo Vibe K4 Note (Black, 16GB)",3093
"OnePlus 3 (Graphite, 64 GB)",2889
"OnePlus 3 (Soft Gold, 64 GB)",2522
Huawei P8lite zwart / 16 GB,1895
Samsung Galaxy Express I8730,1889
"Lenovo Vibe K5 (Gold, VoLTE update)",1801
Samsung Galaxy S6 zwart / 32 GB,1669
Nokia 5800 XpressMusic,1518


#the product 'Lenovo Vibe K4 Note (White,16GB) is found to be the most rated product.

2B. Identify the users with most number of reviews.

In [21]:
grouped_multiple2 = phone_df.groupby(['author']).NewScore.agg(['count']).sort_values(by=['count'], ascending=False)
grouped_multiple2.head(20)

,count
author,
Amazon Customer,54600
9.2,43879
Cliente Amazon,13634
e-bit,5948
Client d'Amazon,5500
Amazon Kunde,3307
Anonymous,1946
einer Kundin,1885
einem Kunden,1379


In [22]:
#The customer  named Amazon Customer is having the most number of reviews.

2C. Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final 
dataset

In [23]:
phone_df.shape


(1000000, 3)

In [24]:
copy = phone_df.copy()
copy.shape

(1000000, 3)

In [25]:

data1 = copy
#how many times a product is rated
data1['ProductRatingCount']=data1['NewScore'].groupby(data1['product']).transform('count')
print(data1.head())
#how many times a customer has reviewd

#data1.query('product=="LG Nexus 5 D820 Unlocked Cellphone, 16GB, Black"')
data1['AuthorRatingCount']=data1['NewScore'].groupby(data1['author']).transform('count')

#data1.query('author=="Jose G."')

finalDataset = data1.query('ProductRatingCount > 50 and AuthorRatingCount>50')
#Report the shape of the final dataset
finalDataset.shape



             author                                            product  \
347247      gilay28                                         Nokia 6301   
80368      Stacy G.     Apple iPhone 4S Verizon Cellphone, 16GB, White   
46940     BulldoZer  Samsung Galaxy Note 3 Neo Smartphone (13,94 cm...   
283355  vinod kumar                                         Nokia 5030   
61365       Giorgio  Huawei P9 Plus Smartphone, LTE, Display 5.5'' ...   

        NewScore  ProductRatingCount  
347247      10.0                  62  
80368       10.0                  46  
46940       10.0                  41  
283355       6.0                  67  
61365       10.0                  46  


(129477, 5)

In [26]:
#from 1,000,000 rowed columns we were able to drop the size to 129,478 rows after filtering.

In [27]:
finalDataset.head(20)

,author,product,NewScore,ProductRatingCount,AuthorRatingCount
114568,Amazon Customer,ZTE Z Max 2 16GB Unlocked GSM 4G LTE Quad-Core...,8.0,154,54600
25345,juan,Lenovo Motorola Moto E - Smartphone libre Andr...,8.0,105,77
26725,9.2,Nokia 6131,10.0,440,43879
129841,9.2,HTC Desire 610,10.0,51,43879
350506,Amazon-Kunde,"Samsung Galaxy S5 Smartphone (5,1 Zoll (12,9 c...",8.0,467,378
62555,Michael,LG G2 D800 GSM 4G LTE Unlocked Smartphone with...,8.0,345,448
271105,9.2,Nokia E52,10.0,682,43879
34654,einer Kundin,"Samsung Galaxy S7 Smartphone, 12,9 cm (5,1 Zol...",10.0,731,1885
33005,9.2,LG Optimus One P500,10.0,520,43879
40012,einer Kundin,"Apple iPhone 7 Plus 5,5"" 128 GB",10.0,278,1885


In [28]:
#the dataset is reduced row wise and increased column wise as we have added two extra columns to get the AuthorRatingCount and ProductRatingCount

3. Build a popularity based model and recommend top 5 mobile phones.

In [29]:
finalDataset.head(10)

,author,product,NewScore,ProductRatingCount,AuthorRatingCount
114568,Amazon Customer,ZTE Z Max 2 16GB Unlocked GSM 4G LTE Quad-Core...,8.0,154,54600
25345,juan,Lenovo Motorola Moto E - Smartphone libre Andr...,8.0,105,77
26725,9.2,Nokia 6131,10.0,440,43879
129841,9.2,HTC Desire 610,10.0,51,43879
350506,Amazon-Kunde,"Samsung Galaxy S5 Smartphone (5,1 Zoll (12,9 c...",8.0,467,378
62555,Michael,LG G2 D800 GSM 4G LTE Unlocked Smartphone with...,8.0,345,448
271105,9.2,Nokia E52,10.0,682,43879
34654,einer Kundin,"Samsung Galaxy S7 Smartphone, 12,9 cm (5,1 Zol...",10.0,731,1885
33005,9.2,LG Optimus One P500,10.0,520,43879
40012,einer Kundin,"Apple iPhone 7 Plus 5,5"" 128 GB",10.0,278,1885


In [30]:
#Since this model has to be built on popularity ie the customer ratings the data is grouped based on most rated products.
finalDataset.groupby('product')['NewScore'].mean().sort_values(ascending=False).head()  

product
3220                                                                                                                                          10.0
Apple iPhone 6S 16GB Black                                                                                                                    10.0
Smartphone Samsung Galaxy S4 Mini Duos Preto com Dual Chip, Tela 4.3", Android 4.2, CÃ¢mera 8MP e Processador Dual Core de 1.7 Ghz - Claro    10.0
Apple iPhone 5c (Blue, 8GB)                                                                                                                   10.0
Nokia Smartphone Nokia Lumia 520 Desbloqueado Oi Windows Phone 8 Tela...                                                                      10.0
Name: NewScore, dtype: float64

In [31]:

final_grouped = finalDataset.groupby('product')['NewScore'].count().sort_values(ascending=False)

final_grouped.head(5)

product
Lenovo Vibe K4 Note (White,16GB)       2190
Lenovo Vibe K4 Note (Black, 16GB)      1827
OnePlus 3 (Graphite, 64 GB)            1303
OnePlus 3 (Soft Gold, 64 GB)           1234
Lenovo Vibe K5 (Gold, VoLTE update)    1126
Name: NewScore, dtype: int64

In [32]:
# The five top products that are recommended will be same for any user as it is a recommendation model based on popularity without considering the individual interest.


4. Build a collaborative filtering model using SVD. You can use SVD from surprise or build it from scratch(Note: Incase you’re building it from scratch you 
can limit your data points to 5000 samples if you face memory issues). Build a collaborative filtering model using kNNWithMeans from surprise. You 
can try both user-based and item-based model

5. Evaluate the collaborative model. Print RMSE value. 
6. Predict score (average rating) for test users. 

In [33]:
#KNNWithMeans
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split


KNN_data= finalDataset[['author', 'product','NewScore']].copy()
KNN_data

# load data
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(KNN_data,reader)
# split the dataset
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)


In [34]:
#item-based
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)
# Predict score (average rating) for test users
test_pred = algo.test(testset)
test_pred
print("Item-based Model : Test Set")
#5. Evaluate the collaborative model. Print RMSE value
print(accuracy.rmse(test_pred, verbose=True))


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 4.0209
4.020948431229767


In [35]:
# user-based
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)
test_pred = algo.test(testset)
print (test_pred)
print("User-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4.0320503190110815

In [36]:
#SVD from Surprise

# load data

SVD_data = finalDataset[['author', 'product','NewScore']].copy()
SVD_data

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(SVD_data,reader)
trainset, testset = train_test_split(data, test_size=.25)
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)



RMSE: 3.9943


3.994276500395522

In [37]:
#9. Try other techniques (Example: cross validation) to get better results. [
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.9796  3.9992  3.9859  3.9748  3.9829  3.9845  0.0082  
MAE (testset)     3.7138  3.7342  3.7183  3.7048  3.7146  3.7171  0.0096  
Fit time          7.89    8.25    8.23    8.03    8.31    8.14    0.16    
Test time         0.36    0.23    0.15    0.34    0.26    0.27    0.08    


{'test_rmse': array([3.97955604, 3.99916094, 3.98587009, 3.97483484, 3.98294678]),
 'test_mae': array([3.71376515, 3.73417678, 3.71827565, 3.70482794, 3.71459716]),
 'fit_time': (7.8896424770355225,
  8.246095657348633,
  8.23477554321289,
  8.033441066741943,
  8.311364889144897),
 'test_time': (0.36397838592529297,
  0.22690606117248535,
  0.15393567085266113,
  0.34392738342285156,
  0.2607612609863281)}

8. Try and recommend top 5 products for test users. 

In [38]:
#Convert List to Dataframe
df_test_pred=pd.DataFrame(test_pred)
#df_test_pred.groupby(df_test_pred['uid'])
df_test_pred

,uid,iid,r_ui,est,details
0,Jose,Motorola Moto G (3Âª GeneraciÃ³n) + Carcasa Of...,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
1,Alessandro,"Alcatel One Touch Idol 3 Smartphone, 4.7"", Dua...",8.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
2,Frank,Huawei P8 Champagne / 16 GB - Overzicht,9.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
3,Amazon Customer,"Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)",2.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
4,Amazon Customer,"Honor 6X (Silver, 32GB)",2.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
...,...,...,...,...,...
38839,9.2,Sony Ericsson P990i,2.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
38840,Anonymous,Samsung Galaxy S7 32GB (Verizon),10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
38841,tony,Asus ZenFone 3 Laser 5.5-inch Glacier Silver [...,2.0,5.0,"{'actual_k': 0, 'was_impossible': False}"
38842,ÐÐ»Ð°Ð´Ð¸Ð¼Ð¸Ñ,Sony Xperia M (Ð±ÐµÐ»ÑÐ¹),9.2,5.0,"{'actual_k': 5, 'was_impossible': False}"


In [39]:
#Sort by uid Ascending and r_ui (rating) descending 
df_test_pred=df_test_pred.sort_values(by = ['uid', 'r_ui'], ascending = [True, False], na_position = 'last')
df_test_pred

,uid,iid,r_ui,est,details
21,9.2,LG Optimus One P500,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
29,9.2,Sony Xperia SL,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
32,9.2,Nokia 6500 Classic,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
43,9.2,Sony Ericsson W580i,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
44,9.2,Samsung S5830i Galaxy Ace Pure White [Android ...,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
...,...,...,...,...,...
20005,Ð®ÑÐ¸Ð¹,Apple iPhone 5s 16GB (ÑÐµÑÑÐ¹ ÐºÐ¾ÑÐ¼Ð¾Ñ),7.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
28265,Ð®ÑÐ¸Ð¹,Sony Xperia ZR (ÑÐµÑÐ½ÑÐ¹),7.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
27016,Ð®ÑÐ¸Ð¹,Apple iPhone 5s 16GB (ÑÐµÑÐµÐ±ÑÐ¸ÑÑÑÐ¹),6.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
4712,Ð®ÑÐ¸Ð¹,Lenovo A2010 (Ð±ÐµÐ»ÑÐ¹),3.0,5.0,"{'actual_k': 5, 'was_impossible': False}"


In [40]:
#Group By uid 
df_test_pred_grouped=df_test_pred.groupby(df_test_pred['uid'])

In [41]:
#In every Group of users Getting the top 5
top_5_recommendations=df_test_pred_grouped.head(n=5)

In [42]:
df_test_pred_grouped.groups

{9.2: [21, 29, 32, 43, 44, 48, 49, 58, 79, 86, 87, 101, 113, 118, 120, 131, 135, 137, 146, 153, 158, 159, 168, 169, 175, 179, 202, 207, 230, 234, 240, 245, 273, 276, 280, 286, 287, 289, 295, 307, 323, 328, 336, 365, 367, 371, 372, 373, 379, 382, 399, 431, 437, 441, 458, 475, 480, 489, 490, 503, 505, 507, 508, 514, 519, 520, 522, 542, 556, 557, 569, 571, 572, 578, 579, 627, 642, 645, 670, 671, 707, 710, 714, 721, 729, 740, 752, 762, 765, 771, 789, 809, 815, 817, 829, 830, 837, 849, 856, 858, ...], '#': [398, 4055, 4211, 7914, 11896, 11954, 15825, 18372, 22170, 22461, 23027, 24843, 25326, 27084, 36135, 37148, 38240, 726, 2788, 7807, 8634, 9189, 10972, 13545, 15229, 17344, 23322, 23820, 28855, 29160, 29331, 29511, 30627, 30832, 32128, 36160, 38659, 15272, 21826, 31363, 32337], '????????': [401, 4388, 4955, 9005, 9053, 13604, 16764, 19432, 19441, 22897, 23546, 24393, 25531, 29893, 31267, 33043, 33612, 34326, 34970, 35039, 3516, 10843, 32599, 37222, 2318, 16129, 25234, 28565, 32896, 7992, 3

In [43]:
#top 5 recommendations for the customer "Albert"
top_5_recommendations.query('uid=="Albert"')

,uid,iid,r_ui,est,details
17390,Albert,Smartphone Samsung Galaxy S III Neo Duos Branc...,10.0,5.0,"{'actual_k': 1, 'was_impossible': False}"
18895,Albert,"HTC Evo 3D Smartphone (10,9 cm (4,3 Zoll) Disp...",10.0,5.0,"{'actual_k': 0, 'was_impossible': False}"
23592,Albert,Microsoft Lumia 950 XL 5.7 inch 32 GB SIM Smar...,10.0,5.0,"{'actual_k': 1, 'was_impossible': False}"
32907,Albert,Samsung Galaxy S4 (I9505) - Smartphone libre A...,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
730,Albert,Samsung Galaxy S7 Edge zwart / 32 GB,9.0,5.0,"{'actual_k': 5, 'was_impossible': False}"


In [44]:
df_test_pred.query('uid=="Albert"')

,uid,iid,r_ui,est,details
17390,Albert,Smartphone Samsung Galaxy S III Neo Duos Branc...,10.0,5.0,"{'actual_k': 1, 'was_impossible': False}"
18895,Albert,"HTC Evo 3D Smartphone (10,9 cm (4,3 Zoll) Disp...",10.0,5.0,"{'actual_k': 0, 'was_impossible': False}"
23592,Albert,Microsoft Lumia 950 XL 5.7 inch 32 GB SIM Smar...,10.0,5.0,"{'actual_k': 1, 'was_impossible': False}"
32907,Albert,Samsung Galaxy S4 (I9505) - Smartphone libre A...,10.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
730,Albert,Samsung Galaxy S7 Edge zwart / 32 GB,9.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
28186,Albert,"MICROSOFT Lumia 650 - 16 GB, Black",9.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
5207,Albert,Huawei P8 grijs / 16 GB,8.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
17063,Albert,Huawei P8 grijs / 16 GB,8.0,5.0,"{'actual_k': 5, 'was_impossible': False}"
